In [366]:
class IntcodeComputer:
    def __init__(self, bits):
        self.bits = bits.copy() + [0]* 1000
        self.i = 0
        self.outputs = []
        self.inputs = []
        self.input_i = 0
        self.relative_base = 0 
        self.state = 'Not Started'

        
    def _apply_mode(self, instruction, i):
        mode = int(instruction[-2-i])
        loc = self.i + i
        
        if mode == 0:
            return self.bits[self.bits[loc]]
        elif mode == 1:
            return self.bits[loc]
        elif mode ==2:
            return self.bits[self.relative_base + self.bits[loc]]
    
    def _write_mode(self, instruction, i, write):
        if instruction[-2-i] == '0':
            self.bits[self.bits[self.i+i]] = write
        else:
            self.bits[self.bits[self.i+i] + self.relative_base] = write
    
    def run(self, inputs=[]):
        if isinstance(inputs, list):
            self.inputs += inputs
        else:
            self.inputs += [inputs]

        while True:
            instruction = ("00000" + str(self.bits[self.i]))[-6:]
            opcode = int(instruction[-2:])
                        
            if opcode == 99:
                self.state = 'Finished'
                return int(self.outputs[-1])
            elif opcode == 1:
                p1 = self._apply_mode(instruction, 1)
                p2 = self._apply_mode(instruction, 2)
                
                self._write_mode(instruction, 3, p1+p2)
                self.i = self.i+4
            elif opcode == 2:
                p1 = self._apply_mode(instruction, 1)
                p2 = self._apply_mode(instruction, 2)
                
                self._write_mode(instruction, 3, p1*p2)
                self.i = self.i+4
            elif opcode == 3:
                if len(self.inputs) <= self.input_i:
                    self.state = 'Paused'
                    if len(self.outputs) > 0:
                        return int(self.outputs[-1])
                    else:
                        return None
                
                self._write_mode(instruction, 1, self.inputs[self.input_i])                
                self.input_i += 1
                self.i = self.i+2
            elif opcode == 4:
                output_val = self._apply_mode(instruction, 1)
                self.outputs += [output_val]
                self.i = self.i+2
            elif opcode == 5:
                p1 = self._apply_mode(instruction, 1)
                p2 = self._apply_mode(instruction, 2)

                if p1 > 0:
                    self.i = p2
                else:
                    self.i = self.i+3
            elif opcode == 6:
                p1 = self._apply_mode(instruction, 1)
                p2 = self._apply_mode(instruction, 2)

                if p1 == 0:
                    self.i = p2
                else:
                    self.i = self.i+3
            elif opcode == 7:
                p1 = self._apply_mode(instruction, 1)
                p2 = self._apply_mode(instruction, 2)
                p3 = self._apply_mode(instruction, 3)
                if p1 < p2:
                    self._write_mode(instruction, 3, 1)
                    #self.bits[self.bits[self.i+3]] = 1
                else:
                    self._write_mode(instruction, 3, 0)
                    #self.bits[self.bits[self.i+3]] = 0
                self.i = self.i+4
            elif opcode == 8:
                p1 = self._apply_mode(instruction, 1)
                p2 = self._apply_mode(instruction, 2)
                p3 = self._apply_mode(instruction, 3)
                if p1 == p2:
                    self._write_mode(instruction, 3, 1)
                    #self.bits[self.bits[self.i+3]] = 1
                else:
                    self._write_mode(instruction, 3, 0)
                    #self.bits[self.bits[self.i+3]] = 0
                self.i = self.i+4            
            elif opcode == 9:
                self.relative_base = self.relative_base + self._apply_mode(instruction, 1)
                self.i = self.i+2

In [171]:
def Q1(intcode, phase_signal):
    output_signal = 0
    for p in phase_signal:
        c = IntcodeComputer(intcode)
        c.run(p)
        output_signal = c.run(output_signal)
    return output_signal

In [365]:
assert Q1([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [4,3,2,1,0]) == 43210, "Wrong"

In [364]:
assert Q1([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0], [0,1,2,3,4]) == 54321, "Wrong"

In [359]:
assert Q1([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0], [1,0,4,3,2]) == 65210, "Wrong"

In [287]:
with open('data/day9.txt') as fn:
    bits = fn.readline().split(',')
    bits = [int(b) for b in bits]

# Q1

## Tests

In [360]:
C = IntcodeComputer([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99])
C.run() 
assert C.outputs == [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99], "WRONG"

In [361]:
C = IntcodeComputer([1102,34915192,34915192,7,4,7,99,0])
assert len(str(C.run()))==16

In [362]:
C = IntcodeComputer([104,1125899906842624,99])
assert C.run() == 1125899906842624, "WRONG"

## Run

In [367]:
C = IntcodeComputer(bits)
C.run([1])

3454977209

# Q2

In [369]:
C = IntcodeComputer(bits)
C.run(2)

50120